This notbook is used to split the ingredients into multiple columns for easy analysis.

In [1]:
# import dependencies
import pandas as pd
import numpy as np
import re

In [2]:
# import the raw csv
recipes_df_raw = pd.read_csv('technical_list.csv')
recipes_df = recipes_df_raw.copy()
recipes_df_raw.head

<bound method NDFrame.head of                                              title           judge  \
0           Liam Charles’s Yorkshire Pudding Wraps    Liam Charles   
1      Rav Gill’s Churros with Banana Butterscotch    Ravneet Gill   
2                 Liam Charles’s Mega-Choccy Stack    Liam Charles   
3                        Rav Gill’s Tiger Pretzels    Ravneet Gill   
4    Liam Charles’s Chocolate Marshmallow Biscuits    Liam Charles   
..                                             ...             ...   
121                Mary Berry’s Hazelnut Dacquoise  Paul Hollywood   
122                       Mary Berry’s Religieuses  Paul Hollywood   
123              Paul Hollywood’s Apricot Couronne  Paul Hollywood   
124      Mary Berry’s Tuiles with Chocolate Mousse  Paul Hollywood   
125                  Mary Berry’s Charlotte Royale  Paul Hollywood   

            makes              difficulty hands_on_time baking_time  \
0               2             Needs skill       25 mins   

In [3]:
# # Make a function to remove p tags from the list
# def remove_tags(ingredients):
#         clean_ingredients = ingredients.replace('<p>','').replace('</p>','')
#         return clean_ingredients

# # iterate through each ingredient list and remove p tags.
# # no longer needed
# ingredients = recipes_df_raw['ingredients']
# recipes_no_tags = recipes_df_raw.copy()
# recipes_no_tags['ingredients_clean'] = ''
# for i in range(len(ingredients)):
#     ingredient_list = ingredients[i]
#     clean_list = remove_tags(ingredient_list)
#     recipes_no_tags['ingredients_clean'][i] = clean_list
# print(recipes_no_tags['ingredients_clean'])

In [91]:
# Formats for ingredients
# 1g unsalted butter
# 1 tsp vanilla extract
# slivered pistachios
# 2 large eggs
# 2 dill pickles, quartered lengthways
# juice of 1/2 lemon

def split_ingredient(ingredient):
    '''
    The split_ingredients funcion takes an ingredient list with each ingredient inside a <p></p> tag. The 
    '''
    comment = 'none'

    # if the ingredient contains a comment (need to deal when the comment is before the recipe like in 50)
    if ',' in ingredient:
        comment = ingredient.split(',')[1]
        ingredient = ingredient.split(',')[0]

    # if ingredient is eggs
    if 'egg' in ingredient:
        quantity = re.findall('\d+', ingredient)
        measurement = re.findall('\d+(.*) ', ingredient)
        return 'it has eggs yo'
    #if the ingredient contains no numbers


    # if measurement is weight or volume
    elif re.match('^\d+[a-zA-Z]',ingredient) is not None:
        quantity = re.findall('\d+', ingredient)[0]
        measurement = re.findall('^\d+([a-zA-Z]+) ', ingredient)[0]
        item = re.findall('\d+[a-zA-Z]+ (.*)', ingredient)[0]
        
    # if ingredient uses tsp or tbsp
    elif 'tbsp' in ingredient: 
        quantity = ingredient.split(' ',1)[0]
        measurement = ingredient.split(' ',1)[1]
        item = re.findall('tbsp (.*)', ingredient)[0]

    elif 'tsp' in ingredient:
        quantity = ingredient.split(' ')[0]
        measurement = ingredient.split(' ')[1]
        item = re.findall('tsp (.*)', ingredient)[0]


    # if if ingredient is juice of
    else:
        return ingredient
    
    ingredient_list_split = [quantity, measurement, item, comment]
    return ingredient_list_split

In [5]:
# still need to deal with line 22 in the csv with the <br>
def seperate_ingredients(ingredient_string):
    '''
    The seperate_ingredients funcion takes an ingredient list with each ingredient inside a <p></p> tag and returns a list containing all the ingredients
    '''
    ingredient_list = re.findall('<p>(.*?)</p>', ingredient_string)
    return ingredient_list

In [19]:
test_ingredient = '200ml whole milk'
print(split_ingredient(test_ingredient))

['200', 'ml', 'whole milk']


In [93]:
ingredient_list = seperate_ingredients(recipes_df['ingredients'][104])
for i in ingredient_list:
    print(split_ingredient(i))

['150', 'g', 'chilled butter', 'none']
['450', 'g', 'plain flour', 'none']
['110', 'g', 'caster sugar', 'none']
['3', 'tsp', 'baking powder', 'none']
['3', 'tsp', 'baking soda', 'none']
pinch of salt
['260', 'ml', 'whole milk', 'none']
['600', 'g', 'strawberries', 'none']
['300', 'ml', 'double cream', 'none']
['30', 'g', 'icing sugar', 'none']
dash of vanilla essence
icing sugar


In [94]:
recipes_df['ingredients'][104]

'[<p>150g chilled butter</p>, <p>450g plain flour</p>, <p>110g caster sugar</p>, <p>3 tsp baking powder</p>, <p>3 tsp baking soda</p>, <p>pinch of salt</p>, <p>260ml whole milk</p>, <p>600g strawberries</p>, <p>300ml double cream</p>, <p>30g icing sugar</p>, <p>dash of vanilla essence</p>, <p>icing sugar, to dust</p>]'